# Feature selection Techniques:

This notebook consists in detailing the most appropriate features according to different feature selection techniques

## Filter Methods 

Calaculate the correlations between the features and target attributes

### Divide of features into two categories : 
    - Categorical features (Nominal): albumin - sugar - red_blood_cells  - pus_cell  - pus_cell_clumps  - bacteria  - hypertension  - diabetes_mellitus  - coronary_artery_disease  - appetite  - peda_edema  - aanemia  - classification
 
    - Numearical features (Ordinal) : age  - blood_pressure  - specific_gravity    - blood_glucose_random  - blood_urea  - serum_creatinine  - sodium  - potassium  - hemoglobin  - packed_cell_volume  - white_blood_cell_count  - red_blood_cell_count
    

To divide them I did check the values in the dataset of each feature and the meaning behind it 

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

In [6]:
clean_df = pd.read_csv(r'Data/clean_dataset.csv')
clean_df.head()
target='classification'

In [5]:
cat_features=['albumin' ,
              'sugar' ,
              'red_blood_cells'  ,
              'pus_cell'  ,
              'pus_cell_clumps'  ,
              'bacteria'  ,
              'hypertension'   ,
              'diabetes_mellitus'  ,
              'coronary_artery_disease'  ,
              'appetite'   ,
              'peda_edema'  ,
              'aanemia']

In [4]:
num_features=['age'  ,
              'blood_pressure'  ,
              'specific_gravity'    ,
              'blood_glucose_random'  ,
              'blood_urea'  ,
              'serum_creatinine'  ,
              'sodium'  ,
              'potassium'  ,
              'hemoglobin'  ,
              'packed_cell_volume',
              'white_blood_cell_count'  ,
              'red_blood_cell_count']

### Correlation between numerical features (Pearson)

In [ ]:
corr=clean_df[num_features].corr()

plt.figure(figsize=(16, 6))
sns.heatmap(corr, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
# strongly  correlated feature are 0.7 and more so we will select the thershold = 0.7
correlation_threshold=0.7

mask = (abs(corr) > correlation_threshold) & (abs(corr) <1)
strongly_correlated_features = corr[mask].stack().dropna().reset_index()

selected_features = set(strongly_correlated_features['level_0']).union(strongly_correlated_features['level_1'])

strongly_correlated_features

In [ ]:
selected_features

In [12]:
def maximizeCorrelation(corr,selected_features):
    max=0
    best_feature_tokeep=''
    for feature in selected_features:
        s= corr[corr['level_0']==feature][0].sum()
        if s > max:
            max=s
            best_feature_tokeep=feature
    return [best_feature_tokeep,max]

In [ ]:
maximizeCorrelation(strongly_correlated_features,selected_features)

#### Conclusion:
- We can delete 'packed_cell_volume' and 'red_blood_cell_count' features and keep the 'hemoglobin' feature, as it maximizes the desired outcome.
- We need to first check the correlation of these features with the target variable before deciding whether to delete them.

### Correlation between numerical features and  the target (categorical variable) 

In [ ]:
corr=[]
for feature in num_features:
    pb_corr, pb_p_value = stats.pointbiserialr(
        clean_df[feature],
        clean_df[target])
    print(pb_p_value)
    corr.append([feature,abs(pb_corr)])
corr=pd.DataFrame(corr,columns=['num_feature','correlation with the target'])

corr.sort_values(by='correlation with the target'  ,ascending=False)

Conclusion:
- hemoglobin, specific_gravity ,packed_cell_volume 	,red_blood_cell_count 

# RQ:
- Can add LDA (the model finds linear combinations of the features that achieve maximum separability between the classes and minimum variance within each class)
- PCA as preprocessing step